In [1]:
import json
import pandas as pd
import numpy as np
import os
from pandas import json_normalize
import ast

Cargar los datos para crear el Dataframe

In [2]:
ruta_archivo_json = r'australian_users_items.json'

# Lee el archivo JSON y carga los datos en una lista de diccionarios
data_list = []
with open(ruta_archivo_json, 'rb') as file:
    for line in file:
        decode_line = line.decode('utf-8')
        data_list.append(ast.literal_eval(decode_line.strip()))

a= pd.DataFrame(data_list)
    

In [5]:
a

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [6]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [7]:
a.isnull().sum()

user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64

brincar hasta T2UserForGender si estas trabajando esa parte

#  T1PlaytimeGenre
apartir de aqui se desanidar con normalizar sin user_id para T1PlaytimeGenre

In [8]:
lista_dataframes = []

# Itera sobre los elementos de la columna 'items'
for fila in a['items']:
    # Normaliza la fila actual y agrega el DataFrame resultante a la lista
    df_normalizado = pd.json_normalize(fila)
    lista_dataframes.append(df_normalizado)

In [9]:
#junto el listado de generos con el dataframe, eliminando genero por la lista de generos
data_items_horas = pd.concat(lista_dataframes, ignore_index=False)
data_items_horas 

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6,0
1,20,Team Fortress Classic,0,0
2,30,Day of Defeat,7,0
3,40,Deathmatch Classic,0,0
4,50,Half-Life: Opposing Force,0,0
...,...,...,...,...
2,346330,BrainBread 2,0,0
3,373330,All Is Dust,0,0
4,388490,One Way To Die: Steam Edition,3,3
5,521570,You Have 10 Seconds 2,4,4


In [14]:
data_user_items_horas = pd.concat(lista_dataframes, ignore_index=False)
data_user_items_horas 

,item_id,item_name,playtime_forever,playtime_2weeks
0,10,Counter-Strike,6,0
1,20,Team Fortress Classic,0,0
2,30,Day of Defeat,7,0
3,40,Deathmatch Classic,0,0
4,50,Half-Life: Opposing Force,0,0
...,...,...,...,...
2,346330,BrainBread 2,0,0
3,373330,All Is Dust,0,0
4,388490,One Way To Die: Steam Edition,3,3
5,521570,You Have 10 Seconds 2,4,4


adecuar los valores: borrar la columna playtime_2weeks,cambiar a mayusculas los datos de item_name, cambiar los minutos a horas en el playtime_forever, eliminar los juegos no jugados

In [10]:
#borrar la columna playtime_2weeks
data_items_horas.drop(['playtime_2weeks'], axis=1, inplace=True)
#cambiar a mayusculas los datos de item_name
data_items_horas['item_name'] = data_items_horas['item_name'].str.upper()
#cambiar los minutos a horas en el playtime_forever
data_items_horas['playtime_forever']= data_items_horas['playtime_forever']//60
#eliminar los juegos no jugados
data_items_horas=data_items_horas[data_items_horas['playtime_forever']!=0]

data_items_horas

,item_id,item_name,playtime_forever
8,300,DAY OF DEFEAT: SOURCE,78
9,240,COUNTER-STRIKE: SOURCE,30
10,3830,PSYCHONAUTS,5
11,2630,CALL OF DUTY 2,1
12,3900,SID MEIER'S CIVILIZATION IV,5
...,...,...,...
4,442080,RIDERS OF ICARUS,2
315,273350,EVOLVE STAGE 2,1
1,730,COUNTER-STRIKE: GLOBAL OFFENSIVE,75
2,346110,ARK: SURVIVAL EVOLVED,10


In [ ]:
# se guarda las columnas ['item_name','item_id'] porque al sumar y agrupar estos se pierden
saved= data_items_horas[['item_name','item_id']]
saved = pd.DataFrame(saved)
saved.drop_duplicates(inplace=True)
saved.reset_index(drop=True, inplace=True)
saved

In [ ]:
#agrupar por itme_id y sumar las horas de juego
data_items_horas = data_items_horas.groupby('item_id')['playtime_forever'].sum().reset_index()
data_items_horas

In [ ]:
data_items_horas[data_items_horas['item_id']=='300']

In [ ]:
#merger item_name en data_items_horas 
df_merged = pd.merge(data_items_horas, saved, on='item_id', how='inner')

In [ ]:
df_merged

In [ ]:
#ya limpios se guardan para el T1PlayTimeGenre
df_merged.to_json('Clean_Players.json',index=False)

# T2UserForGenre  
    crear una lista para el T2UserForGenre


In [4]:
a.head(1)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


In [7]:
#eliminar columnas inncesarias
a.drop(columns=['items_count', 'steam_id', 'user_url'],inplace=True)
a.head(3)

,user_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [11]:
lista_dataframes = []

# Itera sobre los elementos de la columna 'items'
for user_id, fila in zip(a['user_id'], a['items']):
    # Normaliza la fila actual y agrega el DataFrame resultante a la lista
    df_normalizado = pd.json_normalize(fila)
    
    # Agrega el user_id como una nueva columna al DataFrame
    df_normalizado['user_id'] = user_id
    
    # Agrega el DataFrame resultante a la lista
    lista_dataframes.append(df_normalizado)

In [13]:
# Concatena todos los DataFrames en uno solo
data_items_horas = pd.concat(lista_dataframes, ignore_index=True)
data_items_horas

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6.0,0.0,76561197970982479
1,20,Team Fortress Classic,0.0,0.0,76561197970982479
2,30,Day of Defeat,7.0,0.0,76561197970982479
3,40,Deathmatch Classic,0.0,0.0,76561197970982479
4,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479
...,...,...,...,...,...
5153204,346330,BrainBread 2,0.0,0.0,76561198329548331
5153205,373330,All Is Dust,0.0,0.0,76561198329548331
5153206,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331
5153207,521570,You Have 10 Seconds 2,4.0,4.0,76561198329548331


In [14]:
#borrar la columna playtime_2weeks, item_name
data_items_horas.drop(['playtime_2weeks','item_name'], axis=1, inplace=True)
#cambiar los minutos a horas en el playtime_forever
data_items_horas['playtime_forever']= data_items_horas['playtime_forever']//60
#eliminar los juegos no jugados
data_items_horas=data_items_horas[data_items_horas['playtime_forever']!=0]

data_items_horas

,item_id,playtime_forever,user_id
8,300,78.0,76561197970982479
9,240,30.0,76561197970982479
10,3830,5.0,76561197970982479
11,2630,1.0,76561197970982479
12,3900,5.0,76561197970982479
...,...,...,...
5152677,442080,2.0,76561198320038728
5152993,273350,1.0,76561198320136420
5153000,730,75.0,ArkPlays7
5153001,346110,10.0,ArkPlays7


In [26]:
print(type(data_items_horas['user_id'][8]),
type(data_items_horas['playtime_forever'][8]),
type(data_items_horas['item_id'][8]))

<class 'str'> <class 'numpy.int32'> <class 'str'>


In [24]:
data_items_horas['playtime_forever']=data_items_horas['playtime_forever'].astype(int)
data_items_horas

C:\Users\Luis Meza\AppData\Local\Temp\ipykernel_1676\4269443714.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_items_horas['playtime_forever']=data_items_horas['playtime_forever'].astype(int)


,item_id,playtime_forever,user_id
8,300,78,76561197970982479
9,240,30,76561197970982479
10,3830,5,76561197970982479
11,2630,1,76561197970982479
12,3900,5,76561197970982479
...,...,...,...
5152677,442080,2,76561198320038728
5152993,273350,1,76561198320136420
5153000,730,75,ArkPlays7
5153001,346110,10,ArkPlays7


In [6]:
data_items_horas['user_id'] = data_items_horas['user_id'].str.upper() 
data_items_horas

,item_id,playtime_forever,user_id
8,300,78,76561197970982479
9,240,30,76561197970982479
10,3830,5,76561197970982479
11,2630,1,76561197970982479
12,3900,5,76561197970982479
...,...,...,...
5152677,442080,2,76561198320038728
5152993,273350,1,76561198320136420
5153000,730,75,ARKPLAYS7
5153001,346110,10,ARKPLAYS7


In [7]:
data_items_horas.to_json('Clean_Players_T2.json',index=False)